### Task is to take a look at the following datasets provided by Sprocket Central Pty Ltd and identify all data quality issues.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Let's read the first sheet `Transactions` of our workbook.

In [2]:
df1 = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name=1, header=1, engine='openpyxl')

In [3]:
df1.shape

(20000, 13)

Check few records of `Transactions` sheet

In [4]:
df1.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


Sort the values based on `customer_id` and `transaction_date`

In [5]:
df1 = df1.sort_values(['customer_id', 'transaction_date'])

In [6]:
df1

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
9784,9785,72,1,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.300000,37873.0
13423,13424,2,1,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.620000,38573.0
14485,14486,23,1,2017-03-27,0.0,Approved,Norco Bicycles,Mountain,low,small,688.63,612.880000,33455.0
18969,18970,11,1,2017-03-29,1.0,Approved,Giant Bicycles,Standard,high,medium,1274.93,764.960000,35378.0
3764,3765,38,1,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.510000,39526.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,2463,2,3500,2017-04-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.620000,41245.0
8561,8562,33,3500,2017-08-08,0.0,Approved,Giant Bicycles,Standard,medium,small,1311.44,1167.180000,33888.0
17468,17469,0,5034,2017-01-03,0.0,Approved,OHM Cycles,Road,medium,medium,742.54,667.400024,33879.0
16700,16701,0,5034,2017-01-27,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.299988,35667.0


The first issue here is the <b> `product_first_sold_date` </b>  it's an excel datetime format, called serial date. so let's change that to general date format.

In [7]:
# we can use pd.to_datetime()
df1['product_first_sold_date'] = pd.to_datetime(df1['product_first_sold_date'], unit='s')

In [8]:
# check the dtype of all the columns.
df1.dtypes

transaction_id                      int64
product_id                          int64
customer_id                         int64
transaction_date           datetime64[ns]
online_order                      float64
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date    datetime64[ns]
dtype: object

### Here are some rules that i follow while filling out missing data:
***Remove the rows with missing values:***
One option is to remove the rows that contain missing values. However, this should be done with caution, as removing too many rows can significantly reduce the amount of data available for analysis. If the missing values are only a small percentage of the total dataset, it may be acceptable to remove them.

***Impute missing values:*** Another option is to impute the missing values with estimated or predicted values. There are several methods for imputing missing values, such as mean or median imputation, mode imputation, and regression imputation. Mean or median imputation can be used for numeric columns like standard_cost, while mode imputation can be used for categorical columns.

***Create a separate category for missing values:*** Depending on the type of missing data, you may also choose to create a separate category for the missing values. This is typically done for categorical variables, where the missing value is coded as a separate category, such as "unknown" or "not applicable"


In [9]:
# Now lets check the missing data
df1.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

some of the columns have missing values, we can either drop those columns or Fill in missing data with some value or using an interpolation method such as **`ffill` or `bfill`**.

In [10]:
# start with online_order
df1['online_order'].value_counts()

1.0    9829
0.0    9811
Name: online_order, dtype: int64

In [11]:
# fill the missing data using the back-fill method.
df1['online_order'].bfill(axis='rows', inplace=True)

In [12]:
df1['brand'].value_counts()

Solex             4253
Giant Bicycles    3312
WeareA2B          3295
OHM Cycles        3043
Trek Bicycles     2990
Norco Bicycles    2910
Name: brand, dtype: int64

In [13]:
# fill the missing data using the forward-fill method
df1['brand'].ffill(axis='rows', inplace=True)

In [14]:
df1['product_line'].value_counts()

Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: product_line, dtype: int64

In [15]:
# fill the missing data using the back-fill method
df1['product_line'].bfill(axis='rows', inplace=True)

In [16]:
df1['product_class'].value_counts()

medium    13826
high       3013
low        2964
Name: product_class, dtype: int64

In [17]:
# fill the missing data using the back-fill method
df1['product_class'].bfill(axis='rows', inplace=True)

In [18]:
df1['product_size'].value_counts()

medium    12990
large      3976
small      2837
Name: product_size, dtype: int64

In [19]:
# fill the missing data using the forward-fill method
df1['product_size'].ffill(axis='rows', inplace=True)

round the <b>standard_cost</b> to 2 decimal points. and fill the missing values with the mean of the customer's standard_cost.

In [20]:
df1['standard_cost'] = df1['standard_cost'].apply(lambda x: round(x, 2))

In [21]:
df1['standard_cost'] = df1.groupby('customer_id')['standard_cost'].apply(lambda x:x.fillna(x.mean()))

Let's do the same for `product_first_sold_date` and impute with the mean

In [22]:
df1['product_first_sold_date'].isnull().sum()

197

In [23]:
df1['product_first_sold_date'] = df1.groupby('customer_id')['product_first_sold_date'].apply(lambda x: x.fillna(x.mean()))

In [24]:
df1.isnull().sum()

transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
product_first_sold_date    0
dtype: int64

In [25]:
df1

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
9784,9785,72,1,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,1970-01-01 10:31:13
13423,13424,2,1,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,1970-01-01 10:42:53
14485,14486,23,1,2017-03-27,0.0,Approved,Norco Bicycles,Mountain,low,small,688.63,612.88,1970-01-01 09:17:35
18969,18970,11,1,2017-03-29,1.0,Approved,Giant Bicycles,Standard,high,medium,1274.93,764.96,1970-01-01 09:49:38
3764,3765,38,1,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,1970-01-01 10:58:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,2463,2,3500,2017-04-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,1970-01-01 11:27:25
8561,8562,33,3500,2017-08-08,0.0,Approved,Giant Bicycles,Standard,medium,small,1311.44,1167.18,1970-01-01 09:24:48
17468,17469,0,5034,2017-01-03,0.0,Approved,OHM Cycles,Road,medium,medium,742.54,667.40,1970-01-01 09:24:39
16700,16701,0,5034,2017-01-27,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,1970-01-01 09:54:27


Data looks clean for analysis lets save it to a CSV file.

In [26]:
df1.to_csv('TASK 1-Transactions.csv', index=False)